## 1. セットアップ

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 自作モジュールの読み込み
sys.path.append(os.path.abspath('..'))
from configs.config import *
from src.runner import Runner
from src.model_resnet_knn import ModelResNet50KNN  # KNN専用クラス
from src.util import Logger, Util, Metric

print(f"Input dir: {DIR_INPUT}")
print(f"Model dir: {DIR_MODEL}")
print(f"Submission dir: {DIR_SUBMISSIONS}")

## 2. ロガーの設定

In [ ]:
# ロガーの初期化
logger = Logger(path=DIR_LOG)

def get_run_name(model_type):
    """run名の作成"""
    run_name = model_type
    suffix = '_' + datetime.now().strftime("%Y%m%d%H%M")
    run_name = run_name + suffix
    return run_name

logger.info("Logger initialized")

## 3. データ読み込み

In [ ]:
# メタデータ読み込み
df_train = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'train_meta.csv'))
df_test = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'test_meta.csv'))

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")

# ✅ リークしないCV戦略: quarter_sessionでグループ化
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"\n【CV Strategy】")
print(f"  Group column: 'group' (quarter_session)")
print(f"  Total groups: {df_train['group'].nunique()}")
print(f"  Avg samples per group: {len(df_train) / df_train['group'].nunique():.1f}")
print(f"\n  Top 5 groups by size:")
print(df_train['group'].value_counts().head())

# データの確認
print(f"\n【Data Overview】")
print(df_train.head())
print(f"\nLabel distribution:")
print(df_train['label_id'].value_counts().sort_index())

In [ ]:
# ラベル分布確認
label_counts = df_train['label_id'].value_counts().sort_index()
print("\n=== Label Distribution ===")
print(label_counts)

plt.figure(figsize=(10, 5))
label_counts.plot(kind='bar')
plt.title('Training Data Label Distribution')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 4. パラメータ設定（KNN特化）

In [ ]:
# メモ
memo = "ResNet50 + KNN method (k=5)"

# run_name
run_name = get_run_name(model_type="resnet50_knn")

print(f"Run name: {run_name}")
print(f"Memo: {memo}")

In [ ]:
# KNNモデルパラメータ
params = {
    'model_name': 'resnet50',  # 'resnet50' or 'efficientnet_b0'
    
    # KNN固有パラメータ
    'k': 5,  # Top-k近傍数（実験対象）
    
    # 閾値設定
    'threshold': 0.5,  # 1位の類似度閾値
    'min2_threshold': 0.3,  # 2位の類似度閾値
    
    # バッチサイズ
    'batch_size': 32,
    'num_workers': 4,
    
    # キャッシュ設定
    'use_cache': True,  # 特徴量キャッシュを使用
}

print("\n=== KNN Parameters ===")
print(f"  k (neighbors): {params['k']}")
print(f"  threshold: {params['threshold']}")
print(f"  min2_threshold: {params['min2_threshold']}")
print(f"  batch_size: {params['batch_size']}")
print(f"  use_cache: {params['use_cache']}")

## 5. CV戦略設定

In [ ]:
# CV設定
cv_strategy = {
    'method': 'GroupKFold',  # グループ単位で分割
    'n_splits': 5,
    'shuffle': False,
    'random_state': 42
}

print("\n=== CV Strategy ===")
print(f"  Method: {cv_strategy['method']}")
print(f"  N splits: {cv_strategy['n_splits']}")
print(f"  Group column: 'group' (quarter_session)")

## 6. Runner作成

In [ ]:
# Runnerの作成（KNN専用モデルクラスを指定）
runner = Runner(
    run_name=run_name,
    model_cls=ModelResNet50KNN,  # KNN専用クラス
    params=params,
    cv_strategy=cv_strategy,
    logger=logger
)

print(f"\nRunner created: {run_name}")
print(f"Model class: {ModelResNet50KNN.__name__}")

## 7. 5-Fold CV学習

In [ ]:
# 5-fold CV学習
# ⚠️ 初回実行時は特徴抽出に時間がかかります（20-30分程度）
# 2回目以降はキャッシュを使用するため高速です（数分）

scores = runner.train_cv(
    df_train, 
    groups=df_train['group'].values
)

print("\n" + "="*80)
print(f"CV Score (Macro F1): {np.mean(scores):.5f} ± {np.std(scores):.5f}")
print("="*80)

## 8. テストデータ予測

In [ ]:
# CV全モデルのアンサンブル予測
# 各foldモデルの類似度を平均して最終予測
pred_test = runner.predict_cv(df_test)

print(f"\nTest predictions shape: {pred_test.shape}")
print(f"\nPrediction distribution:")
print(pred_test['label_id'].value_counts().sort_index())

In [ ]:
# 予測分布の可視化
plt.figure(figsize=(10, 5))
pred_test['label_id'].value_counts().sort_index().plot(kind='bar')
plt.title('Test Prediction Distribution (KNN)')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.axhline(y=len(pred_test)/11, color='r', linestyle='--', alpha=0.5, label='Uniform')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 9. 提出ファイル作成

In [ ]:
# 提出ファイル保存
submission_path = runner.save_submission(pred_test, run_name)

print(f"\nSubmission file saved: {submission_path}")
print(f"\nFirst 10 predictions:")
print(pred_test.head(10))

## 10. KNN特有の実験: kのチューニング

In [ ]:
# kの値を変えて予測（類似度キャッシュを再利用するため高速）
# ⚠️ 事前に predict_cv() を実行してキャッシュを作成しておく必要あり

k_values = [3, 5, 7, 10, 15]
k_results = []

print("\n=== Testing different k values ===")
print("(Using cached similarities for fast tuning)\n")

for k in k_values:
    # 各foldのモデルでkを変更して予測
    predictions_list = []
    
    for i_fold in range(runner.n_splits):
        # モデルロード
        model = runner.load_model_cv(i_fold)
        
        # キャッシュされた類似度を使って異なるkで予測
        if hasattr(model, 'test_similarities') and model.test_similarities is not None:
            pred = model.predict_with_custom_threshold(
                threshold=params['threshold'],
                min2_threshold=params['min2_threshold'],
                k=k
            )
            predictions_list.append(pred)
        else:
            print(f"Warning: Fold {i_fold} has no cached similarities")
    
    # アンサンブル（多数決）
    if predictions_list:
        predictions_array = np.array(predictions_list)
        final_pred = []
        for i in range(predictions_array.shape[1]):
            values, counts = np.unique(predictions_array[:, i], return_counts=True)
            final_pred.append(values[np.argmax(counts)])
        
        # unknown数をカウント
        unknown_count = np.sum(np.array(final_pred) == -1)
        unknown_ratio = unknown_count / len(final_pred) * 100
        
        k_results.append({
            'k': k,
            'unknown_count': unknown_count,
            'unknown_ratio': unknown_ratio
        })
        
        print(f"k={k:2d}: unknown={unknown_count:4d} ({unknown_ratio:5.2f}%)")

# 結果をDataFrameに
df_k_results = pd.DataFrame(k_results)
print("\n=== k-value tuning results ===")
print(df_k_results)

In [ ]:
# kとunknown比率の関係を可視化
if len(k_results) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(df_k_results['k'], df_k_results['unknown_ratio'], marker='o', linewidth=2)
    plt.xlabel('k (Number of neighbors)', fontsize=12)
    plt.ylabel('Unknown Ratio (%)', fontsize=12)
    plt.title('KNN: Effect of k on Unknown Predictions', fontsize=14)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

## 11. 閾値チューニング実験

In [ ]:
# 閾値を変えて予測（類似度キャッシュを再利用）
threshold_values = np.arange(0.3, 0.8, 0.05)
threshold_results = []

print("\n=== Testing different threshold values ===")
print(f"(Using k={params['k']}, min2_threshold={params['min2_threshold']})\n")

for threshold in threshold_values:
    predictions_list = []
    
    for i_fold in range(runner.n_splits):
        model = runner.load_model_cv(i_fold)
        
        if hasattr(model, 'test_similarities') and model.test_similarities is not None:
            pred = model.predict_with_custom_threshold(
                threshold=threshold,
                min2_threshold=params['min2_threshold'],
                k=params['k']
            )
            predictions_list.append(pred)
    
    # アンサンブル
    if predictions_list:
        predictions_array = np.array(predictions_list)
        final_pred = []
        for i in range(predictions_array.shape[1]):
            values, counts = np.unique(predictions_array[:, i], return_counts=True)
            final_pred.append(values[np.argmax(counts)])
        
        unknown_count = np.sum(np.array(final_pred) == -1)
        unknown_ratio = unknown_count / len(final_pred) * 100
        
        threshold_results.append({
            'threshold': threshold,
            'unknown_count': unknown_count,
            'unknown_ratio': unknown_ratio
        })
        
        print(f"threshold={threshold:.2f}: unknown={unknown_count:4d} ({unknown_ratio:5.2f}%)")

df_threshold_results = pd.DataFrame(threshold_results)
print("\n=== Threshold tuning results ===")
print(df_threshold_results)

In [ ]:
# 閾値とunknown比率の関係を可視化
if len(threshold_results) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(df_threshold_results['threshold'], df_threshold_results['unknown_ratio'], 
             marker='o', linewidth=2, color='orange')
    plt.xlabel('Threshold', fontsize=12)
    plt.ylabel('Unknown Ratio (%)', fontsize=12)
    plt.title('KNN: Effect of Threshold on Unknown Predictions', fontsize=14)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

## 12. Prototype法との比較

### KNN法の特徴
- ✅ **長所**:
  - 訓練データ全体の情報を活用
  - クラス内の多様性を保持
  - kで柔軟に調整可能
  
- ❌ **短所**:
  - 推論が遅い（24,920個との類似度計算）
  - メモリ使用量が多い
  - Prototype法より精度が低い傾向（今回のデータでは）

### Prototype法の特徴
- ✅ **長所**:
  - 推論が高速（11クラスとの比較のみ）
  - メモリ効率が良い
  - クラス代表ベクトルで解釈性が高い
  
- ❌ **短所**:
  - クラス内の多様性を平均化してしまう
  - 外れ値の影響を受ける

### 推奨
- **ベースライン**: Prototype法（高速・シンプル）
- **精度追求**: Ensemble（Prototype + KNN）
- **大規模データ**: ArcFace等のmetric learning

## 13. まとめ

このノートブックでは以下を実施しました：

1. ✅ KNN法での5-fold CV学習
2. ✅ テストデータ予測
3. ✅ kのチューニング実験（類似度キャッシュ活用で高速）
4. ✅ 閾値チューニング実験
5. ✅ Prototype法との比較検討

### 次のステップ
- [ ] EfficientNet-B0で特徴抽出
- [ ] Data Augmentationの導入
- [ ] Prototype + KNNのアンサンブル
- [ ] ArcFace/TripletLossの実装